# Profiling your code

TODO: Figure out how to add links to other parts of the Mila documentation from within the notebook, to include past headers such as:  
Prerequisites Make sure to read the following sections of the documentation before using this example:

[THIS EXAMPLE](/examples/frameworks/pytorch_setup/index)

* :doc:`/examples/frameworks/pytorch_setup/index`

Figuring out where your code may be performing slower than it needs to can be a contrived process. Fear not! There's ways to go about this.  
In the present minimal example, we'll go through a basic profiling example that'll tackle the following:
- Diagnosing if training or dataloading is the bottleneck in your code
- Using the pytorch profiler to find additional bottlenecks
- WIP Potential avenues for further optimization with torch.compile, additional workers, multiple GPUs, etc.

In [1]:
# Show what we changed about main.py? (the important bits, the added metrics for example.)
!python main.py --n-samples=20 --epochs=1 --skip-training

[07/18/24 16:38:29] INFO     INFO:__main__:Setting up ImageNet        ]8;id=978877;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py\main.py]8;;\:]8;id=611693;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py#60\60]8;;\
Train epoch 0: 100%|████████████████████| 1.00/1.00 [00:00<00:00, 1.36Samples/s]
[07/18/24 16:38:35] INFO     INFO:__main__:epoch 0: samples/s:       ]8;id=911051;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py\main.py]8;;\:]8;id=227099;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py#137\137]8;;\
                             24.31625459165844,updates/s: 0.0,                  
                             val_loss: 53.850, val_accuracy: 0.00%              
{"samples/s": 24.31625459165844, "updates/s": 0.0, "val_loss": 53.849586486816406, "val_accuracy": 0.0}


In [2]:
## Imports, setup and the like
!python main.py --n-samples=20 --epochs=1 

[07/18/24 16:41:29] INFO     INFO:__main__:Setting up ImageNet        ]8;id=74399;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py\main.py]8;;\:]8;id=515519;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py#60\60]8;;\
Train epoch 0: 100%|█| 1.00/1.00 [00:01<00:00, 1.84s/Samples, accuracy=0, loss=7
[07/18/24 16:41:35] INFO     INFO:__main__:epoch 0: samples/s:       ]8;id=450668;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py\main.py]8;;\:]8;id=53134;file:///home/mila/c/cesar.valdez/idt/mila-docs/docs/examples/good_practices/profiling/main.py#137\137]8;;\
                             9.782449438915627,updates/s:                       
                             0.5434694132730904, val_loss: 8.047,               
                             val_accuracy: 0.00%                                
{"samples/s": 9.782449438915627, "updates/s":

In [ ]:
## Throughput with training
Take a look at https://docs.mila.quebec/examples/good_practices/launch_many_jobs/index.html

!srun --pty --gpus=1 --cpus-per-task=8 --mem=16G job.sh --epochs=1 --n-samples=20

Comparing the throughput of the former two cells, we can determine that dataloading was/wasn't the bottleneck.  
Did we leave any money on the table? Let's take a more in-depth look with the pytorch profiler.

In [ ]:
## Basic profiler setup

In [ ]:
## Profiler run

A-ha! [Component]'s utilization seems off. Let's introduce a quick fix.

In [ ]:
## Fix to last bottleneck

#!python main.py --num-batches=20 --epochs=1 --skip-training  --num-workers=8

In [ ]:
## New profiler run, with fixed bottleneck

See? we now have a pretty telling difference in profiler outputs. Can we do any better?

 Show how the output of the profiler changes once this last bottleneck is fixed. Give hints as to how to keep identifying the next bottleneck, and potential avenues for further optimization (for example using something like torch.compile, or more workers, multiple GPUs, etc.)


In [ ]:
## More code changes, potential avenues for improvement.